<font face="黑体" size=4 color=#8B0000> Fund Download & IC</font>

In [1]:
import pandas as pd
import numpy as  np
from datetime import datetime
from WindPy import w
w.start() #w.stop() 终止使用wind
w.isconnected()#判断WindPy是否链接成功

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


True

<font face="黑体" size=4 color=#8B0000> 1、数据下载与IC计算函数</font>

In [2]:
def FundDownload(date_download):
    for D in date_download:
        #下载权益型基金
        w_wset_data1=w.wset("sectorconstituent","date="+D+";sectorid=2001010101000000",usedf=True)#普通股票型基金
        commenstock=w_wset_data1[1]
        w_wset_data2=w.wset("sectorconstituent","date="+D+";sectorid=2001010103000000",usedf=True)#指数增强型基金
        indexenhancement=w_wset_data2[1]
        w_wset_data3=w.wset("sectorconstituent","date="+D+";sectorid=2001010201000000",usedf=True)#偏股混合型基金
        stockmix=w_wset_data3[1]
    
        #基金整合
        fund_para=commenstock.append([indexenhancement,stockmix],ignore_index=True)#所有主动权益性基金
        fund_para.index=fund_para['wind_code']
        del fund_para['wind_code']
        fund_para['date'] = fund_para['date'].apply(lambda x : x.strftime('%Y-%m-%d'))
        fundcode=list(fund_para.index)#基金代码
 
        #下载基金指标
        fund_values_temp=w.wss(fundcode,"netasset_total,nav,return_q","tradeDate="+D+"; unit=1",usedf=True)
        fund_values=fund_values_temp[1]
        data=pd.concat([fund_para,fund_values],axis=1)
  
        #储存至excel
        name=D+".xlsx" #'D:\各种材料\研究生\python练习\Python_ZhibinGuo\Fund Summer\基金数据\ '
        writer=pd.ExcelWriter(name)
        data.to_excel(writer,float_format='%.5f')
        writer.save()

In [3]:
def InformCoef(factor,R,rank):
    #该函数计算IC值或者RANKIC值
    # 输入：factor 为因子值序列，R为对应的收益率序列，rank(0或1)：0计算IC值，1计算RANKIC值
    #输出：IC值或者RANKIC值
    
    if rank==0:
        temp=pd.concat([scale,share],axis=1)
        corr_mtx=temp.corr()
        corrcoef= corr_mtx.iloc[0,1]
    else:
        rank_factor=factor.rank(method="first",ascending=False)#默认降序排列
        rank_R=R.rank(method="first",ascending=False)#默认降序排列
        temp=pd.concat([rank_factor,rank_R],axis=1)
        corr_mtx=temp.corr()
        corrcoef= corr_mtx.iloc[0,1]
    return corrcoef

<font face="黑体" size=4 color=#8B0000> 2、IC值计算</font>

In [5]:
import os
import matplotlib.pyplot as plt
os.chdir("D:\各种材料\研究生\python练习\Python_ZhibinGuo\Fund Summer\基金数据")

In [7]:
#需要下载数据的日期
date_download=[]
for year in range(2007,2020):
    for k in range (1,5):
        if k==1:
            date_download.append(datetime(year,3,31).strftime('%Y%m%d'))
        elif k==2:
            date_download.append(datetime(year,6,30).strftime('%Y%m%d'))
        elif k==3:
            date_download.append(datetime(year,9,30).strftime('%Y%m%d'))
        else:
            date_download.append(datetime(year,12,31).strftime('%Y%m%d'))  

#下载数据到指定文件夹
FundDownload(date_download)      

In [88]:
import os
import matplotlib.pyplot as plt
os.chdir("D:\各种材料\研究生\python练习\Python_ZhibinGuo\Fund Summer\基金数据")
filename= [ i+'.xlsx' for i in date_download]
num_files=len(filename)
IC=[]

for i in range(0,num_files-1):
    pre_file=filename[i]
    aft_file=filename[i+1]
    pre_fund=pd.read_excel(pre_file)
    aft_fund=pd.read_excel(aft_file)
    
    pre_fund.index=pre_fund['wind_code']
    del pre_fund['wind_code']
    del pre_fund['date']
    del pre_fund['RETURN_Q']
    pre_fund=pre_fund.rename(columns={'NAV':'NAV1'})
    
    aft_fund.index=aft_fund['wind_code']
    del aft_fund['wind_code']
    del aft_fund['date']
    del aft_fund['RETURN_Q']
    del aft_fund['NETASSET_TOTAL']
    aft_fund=aft_fund.rename(columns={'NAV':'NAV2'})
    
    D=pd.concat([pre_fund,aft_fund],axis=1)
    D=D.dropna()
    
    IC_temp=InformCoef(D['NETASSET_TOTAL'],D['NAV2']/D['NAV1']-1,1)
    IC.append(IC_temp)

D:\Software Download\Anaconda\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [94]:
IC

[0.1959487223149735,
 0.31888949373979314,
 0.07956452029887363,
 0.07515555469938934,
 -0.02575330203413695,
 -0.13696211912199235,
 -0.3280847564147291,
 -0.003970297972540125,
 0.35727817909227205,
 -0.028239319821934897,
 -0.02541893654223595,
 -0.22314407614046602,
 -0.13790680863851595,
 -0.058231287777177794,
 0.04670288509099417,
 0.001982481185269638,
 0.19704160734061066,
 -0.0469360738280712,
 0.029042898081782894,
 0.14499402330738448,
 -0.0817198027754622,
 -0.03368413809461296,
 0.0290572220906379,
 -0.0579680956732216,
 -0.08747755439122828,
 -0.12419876949657946,
 0.022659403534898304,
 -0.2031510429489863,
 -0.03485871239165197,
 -0.23504510777514562,
 -0.19046560324417114,
 0.005626529463076516,
 -0.052032622231217907,
 0.026375571752933653,
 0.017671644788205446,
 -0.2126832742945739,
 -0.06200739322460965,
 -0.060706103877643865,
 -0.1834150475446517,
 -0.1587212359729002,
 -0.008230950409965796,
 -0.14079539730290658,
 0.06878048391615071,
 -0.09564417345155346,
 -